In [2]:
from xgboost import XGBRegressor

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg'

from tqdm import tqdm

from statsmodels.tsa.tsatools import lagmat

import matplotlib.pylab as plt
%matplotlib inline

In [5]:
transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

# transactions.amount = transactions.amount.astype(np.float128)
transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else np.log(x + 1))
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else -x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = (tr_table['day'] - 2) // 7
    tr_table['week_day'] = (tr_table['day'] - 2) % 7
    tr_table['month_num'] = tr_table['day'] // 28
    tr_table['month_day'] = tr_table['day'] % 28
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

train_transactions['log_neg_amount'] = train_transactions.neg_amount.apply(lambda x: np.log(x + 1))
train_transactions['log_pos_amount'] = train_transactions.pos_amount.apply(lambda x: np.log(x + 1))
train_transactions.head()

100%|██████████| 3/3 [00:00<00:00,  1.91it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount
0,4814,0,-1,5,0,0,11098744,0,2365,16.222343,0.0
1,4814,1,-1,6,0,1,7881825,0,1697,15.880070,0.0
2,4814,2,0,0,0,2,6777480,0,1524,15.729116,0.0
3,4814,3,0,1,0,3,9277943,0,1937,16.043151,0.0
4,4814,4,0,2,0,4,9999757,0,1943,16.118071,0.0


In [14]:
train_transactions[train_transactions.neg_amount == 0].shape[0] / train_transactions.shape[0]

0.31986728189515745

In [19]:
submit = pd.read_csv('agile_n_trans_xgb_neg.csv')
submit.volume.quantile(0.25)

100.75219345092773

In [21]:
submit.volume = submit.volume.apply(lambda x: 0 if x<100 else x)
submit.to_csv('hack-submit.csv', index=None)